##### ! pip install pymysql 설치

In [1]:
import import_ipynb

In [2]:
from movie_crawling import data
from weather_crawling import final_data
from melon_crawling import m
from  naver_ranking_news_crawling import df_news
from naver_exchang_crawling import FinallyDf
from Job_opening_crawling import df

importing Jupyter notebook from movie_crawling.ipynb


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))



=====END Movie Crawling=====
importing Jupyter notebook from weather_crawling.ipynb
=====END Weather Crawling=====
importing Jupyter notebook from melon_crawling.ipynb
=====END Melon Crawling=====
importing Jupyter notebook from naver_ranking_news_crawling.ipynb



=====END News Crawling=====
importing Jupyter notebook from naver_exchang_crawling.ipynb
=====END Exchange Crawling=====
importing Jupyter notebook from Job_opening_crawling.ipynb
=====END Job Crawling=====


In [3]:
import pymysql
import pandas as pd
import smtplib, ssl
import numpy as np
import smtplib 
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ipyparams

<IPython.core.display.Javascript object>

1. getConnection()을 통해 mysql과 연결
2. cursor() 메서드를 호출하여 Cursor 객체를 가져오기
3. execute() 메서드를 사용하여 SQL 문장을 DB 서버에 보내기
4. commit() 메서드를 사용하여 데이타를 확정 갱신
5. close() 메서드를 사용하여 DB 연결을 닫는다.

In [4]:
def getConnection():
    con = pymysql.connect(host='localhost', 
                           user = 'root', 
                           password='DB_password',
                           db='DB_name',
                           charset='utf8',
                           cursorclass=pymysql.cursors.DictCursor)
    return con

#####  table  생성 (CREATE 사용)

In [5]:
def make_table(table_name, table_colums):

    conn = getConnection()
    try:
        with conn.cursor() as cursor:
            print('creating table ...')
            
            # 테이블이 생성되지 않은 경우만 테이블 생성
            sql = 'CREATE TABLE IF NOT EXISTS {} {}'.format(table_name, table_colums)
            cursor.execute(sql)
        conn.commit()
    finally:
        conn.close()    

In [6]:
pymysql.converters.encoders[np.float64] = pymysql.converters.escape_float
pymysql.converters.conversions = pymysql.converters.encoders.copy()
pymysql.converters.conversions.update(pymysql.converters.decoders)
pymysql.converters.encoders[np.int64] = pymysql.converters.escape_int
pymysql.converters.conversions = pymysql.converters.encoders.copy()
pymysql.converters.conversions.update(pymysql.converters.decoders)

##### data 입력 ( INSERT 사용)

In [7]:
def input_data(table_name, Data, legth_h):

    conn = getConnection()
    try:
        with conn.cursor() as cursor:
            print('inserting data ...')
            
            sql = "INSERT INTO {} VALUES ({}{})".format(table_name,'%s, '*(legth_h-1),'%s')
            for i in Data.index:
                val = tuple(Data.loc[i,Data.columns])
                cursor.execute(sql, val)
    
        conn.commit()
        print(cursor.rowcount, "record inserted.")
    finally:
        conn.close()    

#####  원하는 날짜의 데이터 선택하기

In [8]:
def data_choose(df,input_date):
    return (df[df['날짜']== input_date])

##### data 조회 (SELECT 사용)

In [9]:
def data_select(table_name, input_date):

    conn = getConnection()
    try:
        with conn.cursor() as cursor:
            sql = "SELECT * FROM {}".format(table_name)
            cursor.execute(sql)
            
            query_result = pd.read_sql (sql, conn)
            return (data_choose(query_result, input_date))

    finally:
        conn.close()

##### 구글 이메일 보내기

In [10]:
def current_notebook_name() :
    notebook_name = ipyparams.notebook_name
    return notebook_name

notebookName = current_notebook_name()

- SSL과 TLS는 사용자와 웹 브라우저 간 통신을 암호화하는데 사용하는 프로토콜
- starttls() : ttl하면 필요한 거
- login() : 구글 이메일 로그인하기(이메일주소, 비밀번호)

In [11]:
def send_email_3(exchange_msg,news_msg,weather_msg,job_msg,movie_msg,music_msg):
    s = smtplib.SMTP('smtp.gmail.com',587)
    s.starttls()
    s.login('@gmail.com','password')
    
    exchange_msg = exchange_msg.to_html(index = False )  # 데이터프레임을 html로 변환
    weather_msg = weather_msg.to_html(index = False )
    news_msg = news_msg.to_html(index = False )
    job_msg = job_msg.to_html(index = False )
    movie_msg = movie_msg.to_html(index = False )
    music_msg = music_msg.to_html(index = False )
    pre =\
        """
        <html>
        <head>
        <style>
        .ft-default-1 { font-size: 24px; }
        .ft-default-2 { font-size: 32px; }
        .ft-smaller { font-size: smaller; }
        .ft-larger { font-size: larger; }
        table {
            width : 90%;
            text-align: center;
            line-height: 1.5;
            margin: 3px 3px;
        }
        table th {
            width: auto;
            padding: 10px;
            font-weight: bold;
            text-align: center;
            vertical-align: top;
            color: #fff;
            background: #ce4869 ;
        }
        table td {
            width: auto;
            vertical-align: top;
            text-align: center;
            border-bottom: 1px solid #ccc;
        }
        </style>
        </head>
        <center><span class="ft-default-2">""" + """{}</span></center>""".format(notebookName)

    one ='<div style="width:50%;float:left;"><table id="tbl1"><caption class="ft-default-1">{}</caption>{}</table></div>'.format("뉴스 속보" , news_msg)
    two ='<div style="width:50%;float:right;"><table id="tbl2"><caption class="ft-default-1">{}</caption>{}</table></div>'.format("날씨",  weather_msg)
    three ='<div style="width:50%;float:left;"><table id="tbl3"><caption class="ft-default-1">{}</caption>{}</table></div>'.format("추천노래",  music_msg)
    four ='<div style="width:50%;float:right;"><table id="tbl4"><caption class="ft-default-1">{}</caption>{}</table></div>'.format("채용정보",  job_msg)
    five ='<div style="width:50%;float:left;"><table id="tbl5"><caption class="ft-default-1">{}</caption>{}</table></div>'.format("환율" , exchange_msg)
    six ='<div style="width:50%;float:right;"><table id="tbl6"><caption class="ft-default-1">{}</caption>{}</table></div>'.format("영화정보" , movie_msg)
    sendHtml = '{}<body><div>{}{}{}{}{}{}</div></body></html>'.format(pre , one , two, three, four, five, six)
    msg = MIMEMultipart()
    
    # 메일 본문 작성
    part1 = MIMEText(sendHtml,'html')
    # 메세지를 메일에 붙여 줍니다.
    msg.attach(part1)
    
    msg['Subject']= '원하는 날짜의 뉴스/날씨/노래/채용/환율/영화정보 결과입니다.'
    # 메일 서버를 이용하여 메일을 발송합니다.  #msg는 object이기 때문에 전송을 위해 .as_string()으로 문자열로 바꿔야함(parsing)
    s.sendmail('@gmail.com','6@gmail.com', msg.as_string())
    
    print('Sent')
    s.quit()

#####  main.py

In [12]:
input_date = input('원하는 날짜를 입력하세요 (2020-05-01 ~ 2020-06-11)  : ')


# file_list = [naver_exchang_crawling.FinallyDf,naver_ranking_news_crawling.FinallyNewsList,weather_crawling.final_data,Job_opening_crawling.df,movie_crawling.data, melon_crawling.m]
Data_list = [FinallyDf, df_news,final_data, df, data, m]
table_name_list = ['exchange','ranking_news','weather','job','movie','music']
table_columns_list =["(날짜 CHAR(10) , USA CHAR(10) NOT NULL, EUR CHAR(10) NOT NULL,JPY CHAR(10) NOT NULL,CNY CHAR(10) NOT NULL)",
                     "(날짜 CHAR(10), title VARCHAR(500) , category VARCHAR(20) ,summary VARCHAR(10000))",
                    "(날짜 CHAR(10) , 최저기온 CHAR(10) NOT NULL, 최고기온 CHAR(10) NOT NULL, 비 CHAR(10) NOT NULL,강수량 CHAR(10) NOT NULL,옷차림 CHAR(100) NOT NULL,날씨 CHAR(10) NOT NULL)",
                    "(날짜 CHAR(30) , 회사 CHAR(100) NOT NULL)", 
                    "(제목 CHAR(50), 날짜 CHAR(10) NOT NULL, 평점 CHAR(10) NOT NULL, 장르 CHAR(50) NOT NULL,국가 CHAR(10) NOT NULL, 상영시간 CHAR(10) NOT NULL, 개봉일 CHAR(100) NOT NULL, 줄거리 VARCHAR(5000) NOT NULL)",
                    "(날짜 CHAR(10) , 노래 CHAR(50) NOT NULL, 가수 CHAR(50) NOT NULL, 좋아요수 CHAR(10) NOT NULL)"
                    ] 

for Data, name, columns in zip(Data_list, table_name_list, table_columns_list):
    table_name = name
    table_columns = columns
    legth_h = len(Data.columns.tolist())
#     print(legth_h)

    # table 생성
    make_table(table_name, table_columns)
    
    # data 입력
    input_data(table_name, Data, legth_h)

    
    # data 조회
    if table_name == 'exchange':
        exchange_msg = data_select(table_name, input_date)
    elif table_name == 'ranking_news':
        news_msg = data_select(table_name, input_date)
    elif table_name == 'weather':
        weather_msg = data_select(table_name, input_date)
    elif table_name == 'job':
        job_msg = data_select(table_name, input_date)
    elif table_name == 'movie':
        movie_msg = data_select(table_name, input_date)
    else:
        music_msg = data_select(table_name, input_date)

# 이메일 보내기
send_email_3(exchange_msg, news_msg, weather_msg, job_msg, movie_msg, music_msg)

원하는 날짜를 입력하세요 (2020-05-01 ~ 2020-06-11)  : 2020-06-08
             날짜       USA       EUR       JPY     CNY
0    2020-02-26  1,215.00  1,322.35  1,100.54  172.88
1    2020-02-27  1,212.00  1,326.35  1,100.77  172.84
2    2020-02-28  1,210.50  1,335.67  1,114.18  173.08
3    2020-03-02  1,194.00  1,323.67  1,107.66  171.38
4    2020-03-03  1,193.00  1,325.84  1,104.58  170.97
..          ...       ...       ...       ...     ...
107  2020-07-02  1,199.00  1,353.49  1,115.61  169.59
108  2020-07-03  1,199.50  1,347.28  1,115.81  169.74
109  2020-07-04  1,199.50  1,347.28  1,115.81  169.74
110  2020-07-05  1,199.50  1,347.28  1,115.81  169.74
111  2020-07-06  1,196.80  1,350.29  1,111.23  169.72

[112 rows x 5 columns]
5
creating table ...
inserting data ...
1 record inserted.
              날짜                                  title category  \
0     2020-05-01            지성호 “김정은 지난 주말 수술 쇼크로 사망했다”       정치   
1     2020-05-01         지성호 "김정은 지난 주말 사망.. 99% 확신" 주장       정치   
2     2020-